### why : To Create of batches of data 

In [30]:
import torch
import pandas as pd
import torch.nn as nn 
import torch.optim as opt
from sklearn.preprocessing import LabelEncoder , StandardScaler , OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [31]:
# Solution - just use two loops one for batch and other for epoch
# Probelm with this solution - Transformation not possibe, shuffling and sampling not possible , paarllel 

# DataSet class
# __init__ --> from where the data is coming from
# __len__ --> give total number of rows
# __getitem__ --> return row of given index , also can apply transformation

# Data Loader class
# shuffle --> make chunks --> pack chunks and make batch --> traning pipeline

 

In [32]:
df = pd.read_csv("Dataset/Titanic/train.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
784,785,0,3,"Ali, Mr. William",male,25.0,0,0,SOTON/O.Q. 3101312,7.0500,NaN,S
324,325,0,3,"Sage, Mr. George John Jr",male,NaN,8,2,CA. 2343,69.5500,NaN,S
197,198,0,3,"Olsen, Mr. Karl Siegwart Andreas",male,42.0,0,1,4579,8.4042,NaN,S
758,759,0,3,"Theobald, Mr. Thomas Leonard",male,34.0,0,0,363294,8.0500,NaN,S
587,588,1,1,"Frolicher-Stehli, Mr. Maxmillian",male,60.0,1,1,13567,79.2000,B41,C


In [33]:
# Preprocss the datset
df = df.drop(columns= ['Name' , 'Cabin' ,"PassengerId" , "Ticket"])

# Split the dataset
y = df['Survived']
X = df.drop(columns= ['Survived'])
X_train , x_test , y_train , y_test = train_test_split(X, y , test_size= 0.2 , random_state= 42)
X_train.shape , x_test.shape

# impuation for Age
Si = SimpleImputer(strategy= "mean")
X_train['Age'] = Si.fit_transform(X_train[['Age']])
x_test['Age'] = Si.transform(x_test[["Age"]])

# Impuation for Embarked
X_train = X_train.fillna({'Embarked': 'missing'})  # Replace with 'missing' instead of dropping
x_test = x_test.fillna({'Embarked': 'missing'})

# OHE
object_columns = X_train.select_dtypes(include=['object']).columns.tolist()
ohe = OneHotEncoder(sparse_output= False)

X_train_ohe_encoder = ohe.fit_transform(X_train[object_columns])
x_test_ohe_encoder = ohe.transform(x_test[object_columns])

X_train_ohe_df = pd.DataFrame(X_train_ohe_encoder , columns= ohe.get_feature_names_out(object_columns), index= X_train.index)
x_test_ohe_df = pd.DataFrame(x_test_ohe_encoder , columns= ohe.get_feature_names_out(object_columns) , index= x_test.index)

X_train = pd.concat([X_train , X_train_ohe_df] , axis= 1)
x_test = pd.concat([x_test , x_test_ohe_df] , axis= 1)

X_train = X_train.drop(object_columns , axis= 1)
x_test = x_test.drop(object_columns , axis= 1)

# Scale Values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(x_test)

# Label Encoder
LE = LabelEncoder()
y_train = LE.fit_transform(y_train)
y_test = LE.transform(y_test)
X_train.shape , y_train.shape


((712, 11), (712,))

In [34]:
# Converting into tensor
import numpy as np 

X_train = np.array(X_train)
x_test = np.array(x_test)
X_train = torch.from_numpy(X_train)
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)
y_train = torch.from_numpy(y_train)

In [35]:
from torch.utils.data import DataLoader , Dataset

class CustomDataSet(Dataset):
    def __init__(self, features , labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, index):
        return self.features[index] , self.labels[index]

In [36]:
dataset =CustomDataSet(X_train , y_train)
dataloader_train = DataLoader(dataset , batch_size= 16 , shuffle= True )

In [37]:
for batch_feartures , batch_lables in dataloader_train: # Create batches od data
    print(batch_feartures.shape)
    print(batch_lables.shape)

torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([16, 11])
torch.Size([16])
torch.Size([

In [38]:
# callate_fn --> in dataloder class , merge data to combine data tensor to create batches
# num_worker --> to run parllel 
# sampler --> how to sample from the data using dataset class

In [39]:
# using Sequentailly container 
# Model Creation
class NeuralNetworkFinal(nn.Module):
    def __init__(self , Feature_count):
        super().__init__() 
        self.network = nn.Sequential(       
        nn.Linear(Feature_count , 5),
        nn.ReLU(),
        nn.Linear( 5 ,3 ),
        nn.ReLU(),
        nn.Linear(3 ,1),
        nn.Sigmoid()
        )

        
    def forward(self , features):
        out = self.network(features)   
        return out

loss_funtion = nn.BCELoss()
    

In [40]:
from torchinfo import summary
lr = 0.1
epoch = 20
X_train = X_train.to(dtype= torch.float32)
x_test = x_test.to(dtype= torch.float32)
y_test = y_test.to(dtype= torch.float32)
y_train = y_train.to(dtype= torch.float32)

# Creating model instance
model = NeuralNetworkFinal(X_train.shape[1])
optimiser = opt.Adam(model.parameters() , lr)
 

summary(model , (712, 11))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetworkFinal                       [712, 1]                  --
├─Sequential: 1-1                        [712, 1]                  --
│    └─Linear: 2-1                       [712, 5]                  60
│    └─ReLU: 2-2                         [712, 5]                  --
│    └─Linear: 2-3                       [712, 3]                  18
│    └─ReLU: 2-4                         [712, 3]                  --
│    └─Linear: 2-5                       [712, 1]                  4
│    └─Sigmoid: 2-6                      [712, 1]                  --
Total params: 82
Trainable params: 82
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.06
Input size (MB): 0.03
Forward/backward pass size (MB): 0.05
Params size (MB): 0.00
Estimated Total Size (MB): 0.08

In [ ]:
# Training pipe line --> here we wil be integrate dataloder class

for ep in range(epoch):
    for batch_feartures , batch_lables in dataloader_train:
        
        batch_feartures = batch_feartures.to(dtype = torch.float32)
        batch_lables =  batch_lables.to(dtype = torch.float32)
        # Forward pass
        y_pred = model(batch_feartures)
        
        # Loss calculation
        loss = loss_funtion(y_pred , batch_lables.view(-1 ,1 ))
        
        # clear gradient 
        optimiser.zero_grad()
        
        # Backprog loss
        loss.backward()
        
        # paramter upadates
        optimiser.step()
        
    print(f"Epoch : {ep+1} , loss : {loss}")
        

Epoch : 1 , loss : 0.5319356918334961
Epoch : 2 , loss : 0.7131079435348511
Epoch : 3 , loss : 0.6126660108566284
Epoch : 4 , loss : 0.7104718089103699
Epoch : 5 , loss : 0.6636423468589783
Epoch : 6 , loss : 0.6813210844993591
Epoch : 7 , loss : 0.6615707278251648
Epoch : 8 , loss : 0.5943147540092468
Epoch : 9 , loss : 0.5768144130706787
Epoch : 10 , loss : 0.8149781227111816
Epoch : 11 , loss : 0.7521846294403076
Epoch : 12 , loss : 0.5862138271331787
Epoch : 13 , loss : 0.5740678310394287
Epoch : 14 , loss : 0.6615975499153137
Epoch : 15 , loss : 0.5695509314537048
Epoch : 16 , loss : 0.6620679497718811
Epoch : 17 , loss : 0.8652433156967163
Epoch : 18 , loss : 0.6619473099708557
Epoch : 19 , loss : 0.6652827262878418
Epoch : 20 , loss : 0.602824866771698


In [42]:
test_dataset = CustomDataSet(x_test , y_test)
dataloader_test = DataLoader(test_dataset , batch_size= 16 , shuffle= True)

In [43]:
# model Evalution
# Evaluate the model

model.eval()
acc_list = []

with torch.no_grad():
    for batch_feartures , batch_lables in dataloader_test:
        
        batch_feartures = batch_feartures.to(dtype = torch.float32)
        batch_lables =  batch_lables.to(dtype = torch.float32)
    
        y_pred = model.forward(batch_feartures)
        y_pred = (y_pred > 0.6).float()  # threshold = 0.6
        
        acc = (y_pred == batch_lables).float().mean() # to calculate accuracy
        acc_list.append(acc)
       
overall_acc = sum(acc_list) / len(acc_list)
print(f"Accuracy: {overall_acc}")
    

Accuracy: 0.569444477558136
